## IMPORT

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")


# Completion Model

In [2]:
#Completion models

from langchain_google_genai import GoogleGenerativeAI

llm_model = GoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.6)

In [3]:
response = llm_model.invoke("write updation sq qurey simple and short no description")
print(response)

for chunk in llm_model.stream("write updation sq qurey simple and short no description"):
    print(chunk , end=" %" , flush=True)

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised NotFound: 404 models/gemini-1.5-flash is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised NotFound: 404 models/gemini-1.5-flash is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods..


KeyboardInterrupt: 

# CHAT MODEL⭐

In [ ]:
#Chatmodel 
from langchain_google_genai import ChatGoogleGenerativeAI

llm_chat_model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.7) 

In [ ]:
from langchain_core.messages import SystemMessage , HumanMessage
message = [
        SystemMessage(content="you are a helpful sql query generator,Just give short answeres & no arguments."),
        HumanMessage(content="write a complex sql query to find the second highest salary from employee table"),
]
response = llm_chat_model.invoke(message)


In [ ]:
response.content

'```sql\nSELECT MAX(salary) FROM employee WHERE salary < (SELECT MAX(salary) FROM employee);\n```'

# Prompt Templete

In [ ]:
from langchain_core.prompts import PromptTemplate # <- completion model support

prompt_templete_completion_model = PromptTemplate.from_template("Write simple SQL query for {action} in table {table}") 

request = prompt_templete_completion_model.format(table="user",action="delete")
response = llm_model.invoke(request)

In [ ]:
request

'Write simple SQL query for delete in table user'

# ChatPrompt Templete⭐

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt_templete_chat_model  = ChatPromptTemplate.from_messages([("system","you are a {profession} specialist , Give short answers like 10 words"),
                                                                ("human","Hello, i am {name}"),
                                                                ("ai","{name} Sure!"),
                                                                ("human","{user_input}")])


request = prompt_templete_chat_model.format(profession="space researcher",user_input="tell about sun",name="sujith")

response = llm_chat_model.invoke(request)

In [ ]:
response


AIMessage(content="Massive, hot plasma sphere; nuclear fusion powerhouse; life's energy source.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []}, id='run--1311f1f9-c868-435b-b151-a1501cf9e4f5-0', usage_metadata={'input_tokens': 41, 'output_tokens': 17, 'total_tokens': 58, 'input_token_details': {'cache_read': 0}})

# FewShort Prompt

In [ ]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate

examples=[{"action":"delete | remote | destory user_sujith","prompt":"write sql query to delete the entire row of the given name user_xxx"},
          {"action":"update | change | modify from user_sujith to user_sappani","prompt":"write sql query to update the entire row of the given name of user_xxx"},
            ]
example_prompt = ChatPromptTemplate.from_messages([("human","{action}"),("ai","{prompt}")])

request = FewShotChatMessagePromptTemplate(example_prompt=example_prompt,examples=examples)

final_prompt = ChatPromptTemplate.from_messages([("system","you are sql query writer."),request,("human","{input}")])
final_prompt
# response = llm_chat_model.invoke(request)

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='you are sql query writer.'), additional_kwargs={}), FewShotChatMessagePromptTemplate(examples=[{'action': 'delete | remote | destory user_sujith', 'prompt': 'write sql query to delete the entire row of the given name user_xxx'}, {'action': 'update | change | modify from user_sujith to user_sappani', 'prompt': 'write sql query to update the entire row of the given name of user_xxx'}], input_variables=[], input_types={}, partial_variables={}, example_prompt=ChatPromptTemplate(input_variables=['action', 'prompt'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['action'], input_types={}, partial_variables={}, template='{action}'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[

# Chains

In [ ]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate

examples=[{"action":"delete | remote | destory user_sujith","prompt":"write sql query to delete the entire row of the given name user_xxx"},
          {"action":"update | change | modify from user_sujith to user_sappani","prompt":"write sql query to update the entire row of the given name of user_xxx"},
            ]
example_prompt = ChatPromptTemplate.from_messages([("human","{action}"),("ai","{prompt}")])

request = FewShotChatMessagePromptTemplate(example_prompt=example_prompt,examples=examples)

final_prompt = ChatPromptTemplate.from_messages([("system","you are sql query writer.write only sql qureirs no descriptions."),request,("human","{input}")])
chain =final_prompt | llm_chat_model

response = chain.invoke({"input":"update the user_xxx to user_sujith_star"})


In [ ]:
for i in chain.stream({"input":"update the user_xxx to user_sujith_star"}):
    print(i.content)

```
sql
UPDATE users SET username = 'user_sujith_star'
 WHERE username = 'user_xxx';
```



# Chain

In [ ]:
from langchain_core.output_parsers.json import SimpleJsonOutputParser
from langchain.prompts import PromptTemplate

prompt_templete = PromptTemplate.from_template("Return a JSON object with 'answer' key that answer the following sql qurstion :{user_input} from database {database},just return the sql query with json object")

json_parser = SimpleJsonOutputParser()
chian = prompt_templete | llm_model | json_parser

result = chain.invoke({"user_input":"add gardening at 12/12/2025","database":"{title:'walking',date:'12/12/2025'}"})

In [ ]:
result

{'answer': "INSERT INTO your_table (title, date) VALUES ('gardening', '2025-12-12');"}

# Advanced Parser⭐

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel , Field
from langchain_core.output_parsers import JsonOutputParser

class structure(BaseModel):
    question : str = Field(description="question to setup the sql query table details short")
    answer : str = Field(description="only sql qurey to the question")
    
parser = JsonOutputParser(pydantic_object=structure)
prompt_templete = PromptTemplate(
    template="Answer the sql questions from user {user_input} via database {database} and {format_instructions} ",
    input_variables=["database","user_input"],
    partial_variables={"format_instructions" : parser.get_format_instructions }
)
chain = prompt_templete | llm_chat_model | parser

request = chain.invoke({"user_input":"Add a gardening at 12/12/2025 ","database":"{Title:'bathing',Date:'12/12/2024'}"} )

In [ ]:
request

{'question': "Add a gardening at 12/12/2025  via database {Title:'bathing',Date:'12/12/2024'}",
 'answer': "```sql\nINSERT INTO your_table (Title, Date) VALUES ('Gardening', '2025-12-12');\n```"}